<a href="https://colab.research.google.com/github/mikosa01/TEAMNOLT/blob/main/group_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import numpy as np 
from pathlib import Path
import sys 
import tensorflow as tf
import pandas as pd 
import matplotlib.pyplot  as plt 
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
downloaded = drive.CreateFile({'id':'1c06v0ViaOvssxGV14zbggB72NcvFR6AE'}) # replace the id with id of file you want to access
downloaded.GetContentFile('Group_Project_Data.zip') 

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
zip_files = ['Train', 'Valid']
for zip_file in zip_files:
  with zipfile.ZipFile('Group_Project_Data.zip'.format(zip_file), 'r') as z: 
    z.extractall('.')
    print('{} unzipped'.format(zip_file))

Train unzipped
Valid unzipped


In [4]:
train_dir =  '/content/Group_Project_Data/Train'
val_dir =  '/content/Group_Project_Data/Valid'

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255)  # rescale pixel values to [0, 1]
val_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # resize images to 224x224 pixels
    batch_size=32,
    class_mode='categorical'  # for multi-class classification
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 6000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [10]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [11]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
history = model.fit(train_generator, epochs=10, validation_data=val_generator)

Epoch 1/10
188/188 [==============================] - 16s 68ms/step - loss: 0.1672 - accuracy: 0.9657 - val_loss: 0.0197 - val_accuracy: 0.9965
Epoch 2/10
188/188 [==============================] - 13s 70ms/step - loss: 0.0086 - accuracy: 0.9982 - val_loss: 0.0182 - val_accuracy: 0.9950
Epoch 3/10
188/188 [==============================] - 12s 63ms/step - loss: 0.0091 - accuracy: 0.9973 - val_loss: 0.0087 - val_accuracy: 0.9965
Epoch 4/10
188/188 [==============================] - 12s 65ms/step - loss: 0.0068 - accuracy: 0.9973 - val_loss: 0.0110 - val_accuracy: 0.9950
Epoch 5/10
188/188 [==============================] - 12s 64ms/step - loss: 0.0038 - accuracy: 0.9987 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 6/10
188/188 [==============================] - 12s 64ms/step - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.0030 - val_accuracy: 0.9995
Epoch 7/10
188/188 [==============================] - 13s 70ms/step - loss: 6.2799e-04 - accuracy: 0.9998 - val_loss: 0.0040 - val_accur